In [1]:
import time

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, Conv2D, Flatten, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

%matplotlib inline

In [3]:
def generator():
    model = Sequential()
    model.add(Dense(20*20*128, activation='relu', input_dim=128))
    model.add(Reshape((20,20,128)))#20
    model.add(Conv2DTranspose(128, (4,4), strides=(3,3), padding='same', activation='relu'))#60
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='relu'))#60
    model.add(Conv2D(3, (20,20), activation='sigmoid', padding='same'))
    return model

In [4]:
def discriminator():
    '''return discriminator model'''
    model = Sequential()
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu', padding='same', input_shape=(120,120,3)))
    model.add(Dropout(0.8))
    model.add(Conv2D(64, (4,4), strides=(3,3), activation='relu', padding='same'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=Adam())
    return model

In [5]:
discriminator().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 64)        3136      
_________________________________________________________________
dropout (Dropout)            (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        65600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 20, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 25601     
Total params: 94,337
Trainable params: 94,337
Non-trainable params: 0
____________________________________________________

In [6]:
generator().summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 51200)             6604800   
_________________________________________________________________
reshape (Reshape)            (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 60, 60, 128)       262272    
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 120, 120, 128)     262272    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 120, 3)       153603    
Total params: 7,282,947
Trainable params: 7,282,947
Non-trainable params: 0
_________________________________________________________________


In [7]:
def generate_latent_vector(n_samples):
    '''generate vector in latent dimension'''
    return np.random.rand(n_samples,128)

In [8]:
def generate_fake_samples(generator, n_samples):
    '''generate fake samples from random numbers'''
    fake_input = generate_latent_vector(n_samples)
    return generator.predict(fake_input), np.zeros((n_samples, 1))

In [9]:
def generate_real_samples(dataset, n_samples):
    mask = np.random.choice(range(dataset.shape[0]),n_samples)
    return dataset[mask,:,:,:], np.ones((n_samples, 1))

In [10]:
def gan(g, d):
    discriminator.trainable = False
    model = Sequential()
    model.add(g)
    model.add(d)
    model.compile(loss='binary_crossentropy',optimizer=Adam())
    return model

In [11]:
def train(g, d, gan, dataset, n_epochs=100, n_batch=256):
    batch_per_epoch = int(dataset.shape[0]/n_batch)
    for i in range(n_epochs):
        for j in range(batch_per_epoch):
            # get real
            t0 = time.time()
            x_real, y_real = generate_real_samples(dataset, int(n_batch/2))
            print("{} seconds to generate real samples".format(time.time()-t0))
            # generate fake
            x_fake, y_fake = generate_fake_samples(g, int(n_batch/2))
            print("{} seconds to generate fake samples".format(time.time()-t0))

            # create training
            x, y = np.vstack((x_real, x_fake)), np.vstack((y_real, y_fake))
            print("{} seconds to stack samples".format(time.time()-t0))

            # update discriminator
            d_loss = d.train_on_batch(x,y)
            print("{} seconds to train d".format(time.time()-t0))

            # generate fake
            x_gan = generate_latent_vector(n_batch)
            y_gan = np.ones((n_batch,1))
            print("{} seconds to generate fake input".format(time.time()-t0))

            # train generator
            gan_loss = gan.train_on_batch(x_gan, y_gan)
            print("{} seconds to train gan".format(time.time()-t0))

            # summarize
            print('>Epoch: %d, %d/%d, d loss=%.3f, gan loss=%.3f' % (i+1, j+1, batch_per_epoch, d_loss, gan_loss))

In [12]:
g = generator()
d = discriminator()
gan = gan(g, d)

In [13]:
dataset = np.load('../data/fusions.npy')

In [14]:
dataset.shape

(22349, 120, 120, 3)

In [15]:
t0 = time.time()
train(g, d, gan, dataset, n_epochs=1, n_batch=2)
time.time() - t0

-0.009629964828491211 seconds to generate real samples
-2.0159339904785156 seconds to generate fake samples
-2.017038106918335 seconds to stack samples
-2.3984100818634033 seconds to train d
-2.398940086364746 seconds to generate fake input
-35.27460598945618 seconds to train gan
>Epoch: 1, 1/11174, d loss=0.762, gan loss=0.759
-0.009538888931274414 seconds to generate real samples
-1.3744819164276123 seconds to generate fake samples
-1.375884771347046 seconds to stack samples
-1.7254300117492676 seconds to train d
-1.7259647846221924 seconds to generate fake input
-17.050557851791382 seconds to train gan
>Epoch: 1, 2/11174, d loss=0.593, gan loss=0.293
-0.004338979721069336 seconds to generate real samples
-1.311683177947998 seconds to generate fake samples
-1.3124120235443115 seconds to stack samples
-1.6737711429595947 seconds to train d
-1.674189805984497 seconds to generate fake input
-19.755353927612305 seconds to train gan
>Epoch: 1, 3/11174, d loss=1.256, gan loss=0.141
-0.0043

KeyboardInterrupt: 